In [66]:
import math
from typing import List
from functools import lru_cache, reduce
from itertools import starmap, accumulate, chain, islice, product, tee
from collections import Counter
from heapq import heapify, heappushpop, heappop, heappush
from operator import add

import arrays
import trees
from importlib import reload
reload(arrays)
reload(trees)
import arrays as ar
import trees as tr
from trees import Node
print()

In [67]:
def parse_grid(s: str):
    nums = map(int, s.split())
    return [list(islice(nums, 9)) for i in range(9)]

In [68]:
grid = parse_grid('3 0 6 5 0 8 4 0 0 5 2 0 0 0 0 0 0 0 0 8 7 0 0 0 0 3 1 0 0 3 0 1 0 0 8 0 9 0 0 8 6 3 0 0 5 0 5 0 0 9 0 6 0 0 1 3 0 0 0 0 2 5 0 0 0 0 0 0 0 0 7 4 0 0 5 2 0 6 3 0 0')
grid

[[3, 0, 6, 5, 0, 8, 4, 0, 0],
 [5, 2, 0, 0, 0, 0, 0, 0, 0],
 [0, 8, 7, 0, 0, 0, 0, 3, 1],
 [0, 0, 3, 0, 1, 0, 0, 8, 0],
 [9, 0, 0, 8, 6, 3, 0, 0, 5],
 [0, 5, 0, 0, 9, 0, 6, 0, 0],
 [1, 3, 0, 0, 0, 0, 2, 5, 0],
 [0, 0, 0, 0, 0, 0, 0, 7, 4],
 [0, 0, 5, 2, 0, 6, 3, 0, 0]]

In [52]:
def at(a, *indexes):
    return reduce(lambda ar, i: ar[i], indexes, a)

In [78]:
def sudoku(grid):
    "Fill in the grid and return True if solved."
    rows, cols, boxes = [0]*9, [0]*9, [0]*9
    left = []

    def flip(n, i, j):
        v = 1 << n
        rows[i] ^= v
        cols[j] ^= v
        boxes[i//3 + 3*(j//3)] ^= v

    for i, j in product(*tee(range(9))):
        flip(grid[i][j] or left.append((i,j)) or 0, i, j)

    def solve(pos):
        i, j = left[pos]
        unsafe = rows[i] | cols[j] | boxes[i//3 + 3*(j//3)]
        for n in range(1, 10):
            if unsafe & (1 << n):
                continue
            flip(n, i, j)
            grid[i][j] = n
            if pos == 0 or solve(pos-1):
                return True
            flip(n, i, j)
            grid[i][j] = 0
        return False
    return solve(len(left)-1)

In [79]:
sudoku(grid)

True

In [80]:
grid

[[3, 1, 6, 5, 7, 8, 4, 9, 2],
 [5, 2, 9, 1, 3, 4, 7, 6, 8],
 [4, 8, 7, 6, 2, 9, 5, 3, 1],
 [2, 6, 3, 4, 1, 5, 9, 8, 7],
 [9, 7, 4, 8, 6, 3, 1, 2, 5],
 [8, 5, 1, 7, 9, 2, 6, 4, 3],
 [1, 3, 8, 9, 4, 7, 2, 5, 6],
 [6, 9, 2, 3, 5, 1, 8, 7, 4],
 [7, 4, 5, 2, 8, 6, 3, 1, 9]]

In [55]:
list(filter(lambda ij: not at(grid, *ij), product(*tee(range(9)))))

[(0, 1),
 (0, 4),
 (0, 7),
 (0, 8),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (2, 0),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (3, 0),
 (3, 1),
 (3, 3),
 (3, 5),
 (3, 6),
 (3, 8),
 (4, 1),
 (4, 2),
 (4, 6),
 (4, 7),
 (5, 0),
 (5, 2),
 (5, 3),
 (5, 5),
 (5, 7),
 (5, 8),
 (6, 2),
 (6, 3),
 (6, 4),
 (6, 5),
 (6, 8),
 (7, 0),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (8, 0),
 (8, 1),
 (8, 4),
 (8, 7),
 (8, 8)]